In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from devito import Grid, Function, TimeFunction, Eq, solve, Operator

In [2]:
# a trivial Operator that, at each time step, increments by 1 all points in the physical domain.
grid = Grid(shape=(4, 4))
u = TimeFunction(name='u', grid=grid, save=3)
op = Operator(Eq(u.forward, u + 1))

In [3]:
# to run op, we have to "apply" it.
summary = op.apply()

Operator `Kernel` ran in 0.01 s


In [4]:
# u has size 3 along the time dimension, since it was built with save=3
# therefore op could only execute 2 timesteps, namely time=0 --> time=1 and time=1 --> time=2
# given Eq(u.forward, u + 1),  executing time=2 --> time=3 would cause out-of-bounds access errors. 
# Devito figures this out automatically and sets appropriate minimum and maximum iteration points.
print(u.dimensions)
print(u.shape)
print(u.data)

(time, x, y)
(3, 4, 4)
[[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [2. 2. 2. 2.]
  [2. 2. 2. 2.]]]


In [5]:
# To access all default arguments used by op without running the Operator, one can run
op.arguments()
# 'u' stores a pointer to the allocated data
# 'timers' stores a pointer to a data structure used for C-level performance profiling

{'u': <cparam 'P' (0x7f8fa2aa2ea0)>,
 'time_m': 0,
 'time_size': 3,
 'time_M': 1,
 'x_m': 0,
 'x_size': 4,
 'x_M': 3,
 'y_m': 0,
 'y_size': 4,
 'y_M': 3,
 'nthreads': 2,
 'h_x': 0.33333334,
 'h_y': 0.33333334,
 'o_x': 0.0,
 'o_y': 0.0,
 'timers': <cparam 'P' (0x7f8fa2de49a0)>}

In [6]:
# One may want to replace some of these default arguments. 
# For example, we could increase the minimum iteration point along the 
# spatial Dimensions x and y, and execute only the very first timestep (t=0)

u.data[:] = 0.  # Explicit reset to initial value
summary = op.apply(x_m=2, y_m=2, time_M=0)
u.data

# x_m and x_M are the minimum and maximum iteration points in the x-dimension
# (similarly for y_m & y_M, z_m & z_M, ... etc. as well as time_m & time_M)

Operator `Kernel` ran in 0.01 s


Data([[[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]],

      [[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 1.]],

      [[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]], dtype=float32)

In [7]:
# the maximum iteration point along the time dimension must be specified whenever save is unset, 
# as in such a case the Operator wouldn't know for how many iterations to run.
v = TimeFunction(name='v', grid=grid)
op2 = Operator(Eq(v.forward, v + 1))
try:
    op2.apply()
except ValueError as e:
    print(e)

No value found for parameter time_M


In [8]:
summary = op2.apply(time_M=4)

Operator `Kernel` ran in 0.01 s


In [9]:
v.data

Data([[[4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.]],

      [[5., 5., 5., 5.],
       [5., 5., 5., 5.],
       [5., 5., 5., 5.],
       [5., 5., 5., 5.]]], dtype=float32)

In [10]:
# The summary variable can be inspected to retrieve performance metrics
summary

PerformanceSummary([(PerfKey(name='section0', rank=None),
                     PerfEntry(time=0.00010399999999999998, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[]))])

In [11]:
# by default Devito avoids to compute performance metrics to minimize the processing time

# To compute all performance metrics, a user could either export the environment variable DEVITO_PROFILING=advanced
# or change the profiling level programmatically before the Operator is constructed

from devito import configuration
configuration['profiling'] = 'advanced'

op = Operator(Eq(u.forward, u*u + 1.))
op.apply()

Operator `Kernel` ran in 0.01 s


PerformanceSummary([(PerfKey(name='section0', rank=None),
                     PerfEntry(time=8.3e-05, gflopss=0.0007710843373493977, gpointss=0.00038554216867469883, oi=0.16666666666666666, ops=2, itershapes=((2, 4, 4),)))])

# Example

An exact traveling wave solution to the 1-dimensional wave-equation,
$$\frac{1}{c^2}\frac{\partial^2 u}{\partial t^2}-\frac{\partial^2 u}{\partial x^2}=0,$$
on the interval $x\in[0,1]$ with $c=1$ is given by:
$$u(x,t)=\left[\mathrm{max}(0,4\zeta(1-\zeta))\right]^{12},$$
where
$$\zeta=4(x-ct)-1,$$
subject to the boundary conditions $u(0,t)=u(1,t)=0$. Compose a Devito operator to solve this problem and check that the numerical solution returns to (approximately) the initial condition at $t=2$.

In [12]:
def compute_zeta(x,t):
    return 4.0*(x-t)-1.0

def compute_u(x,t):
    u1 = 4.0*compute_zeta(x,t)*(1.0-compute_zeta(x,t))
    u2 = np.zeros(u1.shape)
    return (np.maximum(u1,u2))**(12)

In [13]:
# Size of rectangular domain
Lx = 1
# Number of grid points in each direction, including boundary nodes
Nx = 1001
# hence the mesh spacing
dx = Lx/(Nx-1)
x = np.linspace(0, Lx, Nx)

# timestep
dt = 0.4 * dx
print(dx, dt)

0.001 0.0004


In [14]:
# define the grid
grid = Grid(shape=(Nx), extent=(Lx))

# We'll need one of these for composing the boundary conditions
t_s = grid.stepping_dim
time = grid.time_dim

In [15]:
# Wavefield and initial conditions
u = TimeFunction(name='u', grid=grid, time_order=2, space_order=8, save=5001)
u.data[0, :] = compute_u(x, 0)
u.data[1, :] = compute_u(x, dt)

In [16]:
# Define the equation and boundary conditions
pde = u.dt2 - u.dx2
eq = Eq(u.forward, solve(pde, u.forward))
bcs = [Eq(u[t_s+1, 0], 0), Eq(u[t_s+1, -1], 0)]

In [17]:
op = Operator([eq] + bcs)

In [18]:
# Now lets run the operator
time_M = int(2.0 / dt - 1)
print((time_M+1) * dt)
op.apply(time_m=1, time_M=time_M, dt=dt)

2.0


Operator `Kernel` ran in 0.22 s


PerformanceSummary([(PerfKey(name='section0', rank=None),
                     PerfEntry(time=0.2138189999999897, gflopss=0.5382682408953626, gpointss=0.02340296699545055, oi=2.851064499686782, ops=23, itershapes=((4999, 1001),)))])

In [19]:
# create an animation

from matplotlib import animation
from IPython.display import HTML
   
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(6, 6))
ax1 = fig.add_subplot(111, xlim=(0.0, 1.0), ylim=(-1.1, 1.1), xlabel='$x$', ylabel='$u(x)$')
ax1.plot(x, u.data[0, :], 'C1--')
# if we don't close the figure here we get a rogue frame under the movie
plt.close()

line, = ax1.plot([], [], 'C0-', lw=2)
time_text = ax1.text(0.78, 0.92, '', transform=ax1.transAxes)


def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text


def animate(i):
    line.set_data(x, u.data[i, :])
    time_text.set_text('time = {0:.3f}'.format(i * dt))
    return line, time_text


number_frames = 100
frames = np.arange(0, u.shape[0], int(time_M/number_frames)+1)
anim = animation.FuncAnimation(fig, animate, frames, interval=50, blit=True, init_func=init)

HTML(anim.to_jshtml())

In [20]:
eq.evaluate

Eq(u(time + dt, x), dt**2*(-2.84722222*u(time, x)/h_x**2 - 0.00178571429*u(time, x - 4*h_x)/h_x**2 + 0.0253968254*u(time, x - 3*h_x)/h_x**2 - 0.2*u(time, x - 2*h_x)/h_x**2 + 1.6*u(time, x - h_x)/h_x**2 + 1.6*u(time, x + h_x)/h_x**2 - 0.2*u(time, x + 2*h_x)/h_x**2 + 0.0253968254*u(time, x + 3*h_x)/h_x**2 - 0.00178571429*u(time, x + 4*h_x)/h_x**2 + 2.0*u(time, x)/dt**2 - u(time - dt, x)/dt**2))